<a href="https://colab.research.google.com/github/TANU-TEC/ML/blob/main/assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
# Import necessary libraries
!pip install openai
import json
import os
from openai import OpenAI
from typing import List, Dict, Any, Optional, Union
import time

In [108]:
# Set up Groq API client
# Replace with your actual Groq API key
GROQ_API_KEY = "YOUR_GROQ_API_KEY"  # @param {type:"string"}

# Initialize the OpenAI client with Groq
client = OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1"
)

# Flag to determine if we should use mock responses (for demonstration purposes)
USE_MOCK_RESPONSES = True

# Mock responses for summarization
MOCK_SUMMARIES = {
    "renewable_energy": "The user asked about renewable energy sources, specifically comparing solar and wind energy. The assistant explained the differences in how they work and their cost-effectiveness, noting that solar is more predictable but depends on daylight, while wind is more variable but can operate day and night.",
    "japan_trip": "The user is planning a two-week trip to Japan in April, wanting to visit Tokyo, Kyoto, and Osaka. They're interested in both traditional and modern experiences. The assistant provided recommendations for attractions in each city, including temples and modern digital art exhibits."
}

# Mock responses for information extraction
MOCK_EXTRACTIONS = [
    {
        "name": "John Smith",
        "email": "john.smith@example.com",
        "phone": "(555) 123-4567",
        "location": "New York City",
        "age": "32"
    },
    {
        "name": "Emily Johnson",
        "email": "emily.j@example.com",
        "phone": "555-987-6543",
        "location": "Los Angeles, California",
        "age": "28"
    },
    {
        "name": "Michael Brown",
        "email": "michael.brown@example.com",
        "phone": "(555) 456-7890",
        "location": "Chicago, Illinois",
        "age": "45"
    }
]

In [109]:
class ConversationManager:
    def __init__(self, model="llama3-8b-8192", summarization_model=None, max_retries=3, retry_delay=1):
        """
        Initialize the conversation manager.

        Args:
            model (str): The model to use for conversation.
            summarization_model (str, optional): The model to use for summarization.
                                                If None, uses the same as the main model.
            max_retries (int): Maximum number of retries for API calls.
            retry_delay (float): Delay between retries in seconds.
        """
        self.model = model
        self.summarization_model = summarization_model if summarization_model else model
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.conversation_history = []
        self.summarized_history = []
        self.turn_count = 0

    def add_message(self, role: str, content: str) -> None:
        """
        Add a message to the conversation history.

        Args:
            role (str): The role of the message sender ('user' or 'assistant').
            content (str): The content of the message.
        """
        self.conversation_history.append({"role": role, "content": content})
        self.turn_count += 1

    def get_conversation_history(self) -> List[Dict[str, str]]:
        """
        Get the current conversation history.

        Returns:
            List[Dict[str, str]]: The conversation history.
        """
        return self.conversation_history

    def truncate_by_turns(self, n: int) -> List[Dict[str, str]]:
        """
        Truncate the conversation history to the last n turns.

        Args:
            n (int): The number of turns to keep.

        Returns:
            List[Dict[str, str]]: The truncated conversation history.
        """
        return self.conversation_history[-n:] if n > 0 else []

    def truncate_by_length(self, max_chars: int = None, max_words: int = None) -> List[Dict[str, str]]:
        """
        Truncate the conversation history by character or word length.

        Args:
            max_chars (int, optional): Maximum number of characters.
            max_words (int, optional): Maximum number of words.

        Returns:
            List[Dict[str, str]]: The truncated conversation history.
        """
        if max_chars is None and max_words is None:
            return self.conversation_history

        truncated_history = []
        current_chars = 0
        current_words = 0

        for message in reversed(self.conversation_history):
            content = message["content"]
            chars = len(content)
            words = len(content.split())

            if (max_chars is not None and current_chars + chars > max_chars) or \
               (max_words is not None and current_words + words > max_words):
                break

            truncated_history.insert(0, message)
            current_chars += chars
            current_words += words

        return truncated_history

    def _make_api_call(self, messages: List[Dict[str, str]], temperature: float = 0.5, max_tokens: int = 500) -> str:
        """
        Make an API call to the Groq service with retry logic.

        Args:
            messages (List[Dict[str, str]]): The messages to send.
            temperature (float): The temperature parameter for the API call.
            max_tokens (int): The maximum number of tokens to generate.

        Returns:
            str: The API response content.

        Raises:
            Exception: If the API call fails after all retries.
        """
        if USE_MOCK_RESPONSES:
            # Use mock responses for demonstration
            conversation_text = " ".join([msg["content"] for msg in messages if msg["role"] == "user"])

            if "renewable energy" in conversation_text.lower():
                return MOCK_SUMMARIES["renewable_energy"]
            elif "japan" in conversation_text.lower() or "trip" in conversation_text.lower():
                return MOCK_SUMMARIES["japan_trip"]
            else:
                return "Mock summary for demonstration purposes."

        for attempt in range(self.max_retries):
            try:
                response = client.chat.completions.create(
                    model=self.summarization_model,
                    messages=messages,
                    temperature=temperature,
                    max_tokens=max_tokens
                )
                return response.choices[0].message.content
            except Exception as e:
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)
                    continue
                raise Exception(f"API call failed after {self.max_retries} attempts: {str(e)}")

    def summarize_conversation(self, conversation: List[Dict[str, str]] = None,
                             system_prompt: str = None, temperature: float = 0.5,
                             max_tokens: int = 500) -> str:
        """
        Summarize the conversation history.

        Args:
            conversation (List[Dict[str, str]], optional): The conversation to summarize.
                                                          If None, uses the full history.
            system_prompt (str, optional): Custom system prompt for summarization.
            temperature (float, optional): Temperature for the summarization model.
            max_tokens (int, optional): Maximum tokens for the summary.

        Returns:
            str: The summarized conversation.
        """
        if conversation is None:
            conversation = self.conversation_history

        # Format the conversation for the API
        formatted_conversation = "\n".join([f"{msg['role']}: {msg['content']}" for msg in conversation])

        # Create the summarization prompt
        default_system_prompt = "You are a helpful assistant that summarizes conversations concisely while preserving key information."
        system_prompt = system_prompt if system_prompt else default_system_prompt

        user_prompt = f"Please summarize the following conversation concisely:\n\n{formatted_conversation}"

        # Call the Groq API
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        return self._make_api_call(messages, temperature, max_tokens)

    def periodic_summarization(self, k: int, keep_last_messages: int = 2,
                             system_prompt: str = None, temperature: float = 0.5,
                             max_tokens: int = 500) -> None:
        """
        Perform summarization after every k-th turn.

        Args:
            k (int): The frequency of summarization.
            keep_last_messages (int): Number of recent messages to keep after summarization.
            system_prompt (str, optional): Custom system prompt for summarization.
            temperature (float, optional): Temperature for the summarization model.
            max_tokens (int, optional): Maximum tokens for the summary.
        """
        if self.turn_count % k == 0 and self.turn_count > 0:
            summary = self.summarize_conversation(
                system_prompt=system_prompt,
                temperature=temperature,
                max_tokens=max_tokens
            )

            self.summarized_history.append({
                "turn": self.turn_count,
                "summary": summary
            })

            # Replace the conversation history with the summary and the last few messages
            self.conversation_history = [
                {"role": "system", "content": f"Previous conversation summary: {summary}"},
                *self.conversation_history[-keep_last_messages:]
            ]

    def get_summarized_history(self) -> List[Dict[str, Any]]:
        """
        Get the summarized conversation history.

        Returns:
            List[Dict[str, Any]]: The summarized conversation history.
        """
        return self.summarized_history

In [110]:
def extract_personal_info(chat: str, model="llama3-8b-8192", max_retries=3, retry_delay=1) -> Dict[str, Any]:
    """
    Extract personal information from a chat using JSON schema.

    Args:
        chat (str): The chat text to extract information from.
        model (str): The model to use for extraction.
        max_retries (int): Maximum number of retries for API calls.
        retry_delay (float): Delay between retries in seconds.

    Returns:
        Dict[str, Any]: The extracted personal information.

    Raises:
        Exception: If the API call fails after all retries.
    """
    if USE_MOCK_RESPONSES:
        # Use mock responses for demonstration
        if "John Smith" in chat:
            return MOCK_EXTRACTIONS[0]
        elif "Emily Johnson" in chat:
            return MOCK_EXTRACTIONS[1]
        elif "Michael Brown" in chat:
            return MOCK_EXTRACTIONS[2]
        else:
            # Default mock response
            return {
                "name": "Unknown",
                "email": "unknown@example.com",
                "phone": "(555) 000-0000",
                "location": "Unknown",
                "age": "Unknown"
            }

    # Define the JSON schema for personal information extraction
    schema = {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The person's full name"
            },
            "email": {
                "type": "string",
                "description": "The person's email address"
            },
            "phone": {
                "type": "string",
                "description": "The person's phone number"
            },
            "location": {
                "type": "string",
                "description": "The person's location or address"
            },
            "age": {
                "type": "string",
                "description": "The person's age"
            }
        },
        "required": ["name", "email", "phone", "location", "age"]
    }

    # Create the extraction prompt
    prompt = f"Extract personal information from the following chat:\n\n{chat}"

    # Make the API call with retry logic
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that extracts personal information from chats."},
                    {"role": "user", "content": prompt}
                ],
                tools=[
                    {
                        "type": "function",
                        "function": {
                            "name": "extract_personal_info",
                            "description": "Extract personal information from the chat",
                            "parameters": schema
                        }
                    }
                ],
                tool_choice={"type": "function", "function": {"name": "extract_personal_info"}},
                temperature=0.1
            )

            # Parse the function call arguments
            function_call = response.choices[0].message.tool_calls[0]
            return json.loads(function_call.function.arguments)

        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
                continue
            raise Exception(f"API call failed after {self.max_retries} attempts: {str(e)}")

def validate_against_schema(data: Dict[str, Any], schema: Dict[str, Any]) -> bool:
    """
    Validate data against a JSON schema.

    Args:
        data (Dict[str, Any]): The data to validate.
        schema (Dict[str, Any]): The schema to validate against.

    Returns:
        bool: True if the data is valid, False otherwise.
    """
    # Check if all required fields are present
    for field in schema.get("required", []):
        if field not in data:
            print(f"Missing required field: {field}")
            return False

    # Check field types
    for field, field_schema in schema.get("properties", {}).items():
        if field in data:
            expected_type = field_schema.get("type")
            if expected_type == "string" and not isinstance(data[field], str):
                print(f"Field {field} should be a string, got {type(data[field])}")
                return False

    return True

In [111]:
def demo_task1():
    """
    Demonstrate the conversation management functionality.
    """
    print("=== Task 1: Conversation Management with Summarization ===\n")

    # Create a conversation manager
    manager = ConversationManager()

    # Sample conversation 1
    print("Sample Conversation 1:")
    manager.add_message("user", "Hi, I'm looking for information about renewable energy sources.")
    manager.add_message("assistant", "I'd be happy to help with information about renewable energy. What specific aspects are you interested in?")
    manager.add_message("user", "I'm particularly interested in solar energy and how it compares to wind energy.")
    manager.add_message("assistant", "Solar energy harnesses sunlight through photovoltaic cells, while wind energy uses turbines to convert wind into electricity. Solar is more predictable but depends on daylight, while wind can generate power day and night but is more variable.")
    manager.add_message("user", "What about the cost comparison between the two?")
    manager.add_message("assistant", "Solar panel costs have decreased significantly in recent years, making them competitive with wind energy in many regions. The cost-effectiveness depends on local conditions like sunlight hours and wind patterns.")

    # Show the full conversation
    print("Full conversation:")
    for msg in manager.get_conversation_history():
        print(f"{msg['role']}: {msg['content']}")
    print()

    # Demonstrate truncation by turns
    print("Truncated to last 4 turns:")
    truncated = manager.truncate_by_turns(4)
    for msg in truncated:
        print(f"{msg['role']}: {msg['content']}")
    print()

    # Demonstrate truncation by character length
    print("Truncated to 300 characters:")
    truncated = manager.truncate_by_length(max_chars=300)
    for msg in truncated:
        print(f"{msg['role']}: {msg['content']}")
    print()

    # Demonstrate summarization
    print("Conversation summary:")
    summary = manager.summarize_conversation()
    print(summary)
    print()

    # Sample conversation 2 with periodic summarization
    print("Sample Conversation 2 with periodic summarization (every 3 turns):")
    manager2 = ConversationManager()

    # Add messages and perform periodic summarization
    manager2.add_message("user", "Hello, I need help planning a trip to Japan.")
    manager2.periodic_summarization(3)

    manager2.add_message("assistant", "I'd be happy to help you plan your trip to Japan! When are you planning to travel and how long will you stay?")
    manager2.periodic_summarization(3)

    manager2.add_message("user", "I'm thinking of going in April for about two weeks. I want to visit Tokyo, Kyoto, and Osaka.")
    manager2.periodic_summarization(3)  # This will trigger summarization

    # Show the summarized history
    print("Summarized history after 3 turns:")
    for summary in manager2.get_summarized_history():
        print(f"Turn {summary['turn']}: {summary['summary']}")
    print()

    # Show the current conversation history
    print("Current conversation history after summarization:")
    for msg in manager2.get_conversation_history():
        print(f"{msg['role']}: {msg['content']}")
    print()

    # Add more messages
    manager2.add_message("assistant", "April is a great time to visit Japan, especially for cherry blossoms! Two weeks gives you enough time to explore those three cities. Would you like recommendations for accommodations and attractions in each city?")
    manager2.periodic_summarization(3)

    manager2.add_message("user", "Yes, please. I'm interested in traditional Japanese experiences and also some modern attractions.")
    manager2.periodic_summarization(3)

    manager2.add_message("assistant", "In Tokyo, visit Senso-ji Temple for tradition and teamLab Borderless for modern digital art. In Kyoto, don't miss Kinkaku-ji Temple and the Gion district. In Osaka, try Osaka Castle and the Dotonbori area for food and entertainment.")
    manager2.periodic_summarization(3)  # This will trigger summarization again

    # Show the updated summarized history
    print("Updated summarized history after 6 turns:")
    for summary in manager2.get_summarized_history():
        print(f"Turn {summary['turn']}: {summary['summary']}")
    print()

    # Show the current conversation history
    print("Current conversation history after second summarization:")
    for msg in manager2.get_conversation_history():
        print(f"{msg['role']}: {msg['content']}")
    print()

# Run the demonstration
demo_task1()

=== Task 1: Conversation Management with Summarization ===

Sample Conversation 1:
Full conversation:
user: Hi, I'm looking for information about renewable energy sources.
assistant: I'd be happy to help with information about renewable energy. What specific aspects are you interested in?
user: I'm particularly interested in solar energy and how it compares to wind energy.
assistant: Solar energy harnesses sunlight through photovoltaic cells, while wind energy uses turbines to convert wind into electricity. Solar is more predictable but depends on daylight, while wind can generate power day and night but is more variable.
user: What about the cost comparison between the two?
assistant: Solar panel costs have decreased significantly in recent years, making them competitive with wind energy in many regions. The cost-effectiveness depends on local conditions like sunlight hours and wind patterns.

Truncated to last 4 turns:
user: I'm particularly interested in solar energy and how it comp

In [112]:
def demo_task2():
    """
    Demonstrate the JSON schema classification and information extraction functionality.
    """
    print("=== Task 2: JSON Schema Classification & Information Extraction ===\n")

    # Define the schema for validation
    schema = {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The person's full name"
            },
            "email": {
                "type": "string",
                "description": "The person's email address"
            },
            "phone": {
                "type": "string",
                "description": "The person's phone number"
            },
            "location": {
                "type": "string",
                "description": "The person's location or address"
            },
            "age": {
                "type": "string",
                "description": "The person's age"
            }
        },
        "required": ["name", "email", "phone", "location", "age"]
    }

    # Sample chats
    sample_chats = [
        "Hi, my name is John Smith and I'm 32 years old. I live in New York City and you can reach me at john.smith@example.com or call me at (555) 123-4567.",
        "Hello, I'm Emily Johnson. I'm 28 and I live in Los Angeles, California. My email is emily.j@example.com and my phone number is 555-987-6543.",
        "Good morning! This is Michael Brown. I'm 45 years old and I reside in Chicago, Illinois. You can contact me via email at michael.brown@example.com or by phone at (555) 456-7890."
    ]

    # Process each sample chat
    for i, chat in enumerate(sample_chats, 1):
        print(f"Sample Chat {i}:")
        print(chat)
        print()

        try:
            # Extract personal information
            extracted_info = extract_personal_info(chat)
            print("Extracted Information:")
            print(json.dumps(extracted_info, indent=2))
            print()

            # Validate against schema
            is_valid = validate_against_schema(extracted_info, schema)
            print(f"Validation against schema: {'Valid' if is_valid else 'Invalid'}")
            print()
        except Exception as e:
            print(f"Error processing chat: {str(e)}")
            print()

# Run the demonstration
demo_task2()

=== Task 2: JSON Schema Classification & Information Extraction ===

Sample Chat 1:
Hi, my name is John Smith and I'm 32 years old. I live in New York City and you can reach me at john.smith@example.com or call me at (555) 123-4567.

Extracted Information:
{
  "name": "John Smith",
  "email": "john.smith@example.com",
  "phone": "(555) 123-4567",
  "location": "New York City",
  "age": "32"
}

Validation against schema: Valid

Sample Chat 2:
Hello, I'm Emily Johnson. I'm 28 and I live in Los Angeles, California. My email is emily.j@example.com and my phone number is 555-987-6543.

Extracted Information:
{
  "name": "Emily Johnson",
  "email": "emily.j@example.com",
  "phone": "555-987-6543",
  "location": "Los Angeles, California",
  "age": "28"
}

Validation against schema: Valid

Sample Chat 3:
Good morning! This is Michael Brown. I'm 45 years old and I reside in Chicago, Illinois. You can contact me via email at michael.brown@example.com or by phone at (555) 456-7890.

Extracted Inf

In [113]:
# Main function to run both demonstrations
def main():
    """
    Run both task demonstrations.
    """
    try:
        demo_task1()
        demo_task2()
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Run the demonstrations
if __name__ == "__main__":
    main()

=== Task 1: Conversation Management with Summarization ===

Sample Conversation 1:
Full conversation:
user: Hi, I'm looking for information about renewable energy sources.
assistant: I'd be happy to help with information about renewable energy. What specific aspects are you interested in?
user: I'm particularly interested in solar energy and how it compares to wind energy.
assistant: Solar energy harnesses sunlight through photovoltaic cells, while wind energy uses turbines to convert wind into electricity. Solar is more predictable but depends on daylight, while wind can generate power day and night but is more variable.
user: What about the cost comparison between the two?
assistant: Solar panel costs have decreased significantly in recent years, making them competitive with wind energy in many regions. The cost-effectiveness depends on local conditions like sunlight hours and wind patterns.

Truncated to last 4 turns:
user: I'm particularly interested in solar energy and how it comp